In [ ]:
from matplotlib import pyplot as plt
import scanpy as sc
import scanorama as scrama
import numpy as np
import pandas as pd

You need to set this to wherever you downloaded the data from NCBI GEO Datasets, accession [GSE161465](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE161465)

In [ ]:
burkhardt_data_directory = '' 

In [ ]:
burkhardt = sc.read_h5ad(burkhardt_data_directory + 'filtered_counts.h5ad')

In [ ]:
burkhardt_meta = pd.read_csv(burkhardt_data_directory + 'metadata.csv')

In [ ]:
burkhardt_gene_symbols = pd.DataFrame(index=burkhardt_index, data=burkhardt.var.index.str.split(" (|)").str[2].str.split('(').str[1])
burkhardt_gene_symbols.rename(columns={0:'gene_symbols'}, inplace=True)
burkhardt_gene_symbols['gene_symbols'].str.split(')').str[0]

In [ ]:
burkhardt.var = burkhardt_gene_symbols
burkhardt.var_names_make_unique()

In [ ]:
# Normalize and log-transform
burkhardt.layers['counts'] = burkhardt.X.copy() # Store raw counts

# Normalise the data
sc.pp.normalize_total(burkhardt, target_sum=1e4)
sc.pp.log1p(burkhardt)

In [ ]:
# Identify the highly-variable genes. We use the CellRanger routine provided in Scanpy.
target_genes = 2000
sc.pp.highly_variable_genes(burkhardt, flavor='cell_ranger', n_top_genes=target_genes, batch_key='Condition')
sc.pp.pca(burkhardt, use_highly_variable=True)
sc.pp.neighbors(burkhardt, n_neighbors=30)
sc.tl.umap(burkhardt)
sc.pl.umap(burkhardt, color='Condition')

In [ ]:
# As we don't have enough target genes, we need to consider HVGs in all but one batches.
n_batches = len(burkhardt.obs['Donor'].cat.categories)
# These are the genes that are variable across all batches
nbatch1_dispersions = burkhardt.var['dispersions_norm'][burkhardt.var.highly_variable_nbatches > n_batches - 1]
nbatch1_dispersions.sort_values(ascending=False, inplace=True)
print(len(nbatch1_dispersions))

# Fill up the genes now, using this method from the Theis lab
enough = False
hvg = nbatch1_dispersions.index[:]
not_n_batches = 1

# We'll go down one by one, until we're selecting HVGs from just a single gbatch
while not enough:
    
    target_genes_diff = target_genes - len(hvg) # Get the number of genes we still need to fill up
    
    tmp_dispersions = burkhardt.var['dispersions_norm'][burkhardt.var.highly_variable_nbatches == (n_batches - not_n_batches)]
    
    # If we haven't hit the target gene numbers, add this to the list and we repeat this iteration
    if len(tmp_dispersions) < target_genes_diff:
        
        hvg = hvg.append(tmp_dispersions.index)
        not_n_batches += 1
        
    else:
        
        tmp_dispersions.sort_values(ascending=False, inplace=True)
        hvg = hvg.append(tmp_dispersions.index[:target_genes_diff])
        enough = True

In [ ]:
# Subset the data on the HVG to speed things up
burkhardt_hvg = burkhardt[:, hvg] # Filter out genes that do not vary much across cells

In [ ]:
# Split the data into batches (marked by 'Condition')
burkhardt_split = []

for sample in burkhardt_hvg.obs['Condition'].unique():
    burkhardt_split.append(burkhardt_hvg[burkhardt_hvg.obs['Condition']==sample].copy())

In [ ]:
# Now we run Scanorama on the split data. (I find that we don't really need the batch-corrected data for these datasets)
scrama.integrate_scanpy(burkhardt_split, ds_names = list(burkhardt_hvg.obs['Condition'].unique()))

In [ ]:
embeddings = [adata.obsm['X_scanorama'] for adata in burkhardt_split]

# Consider when we just take the embedding
embeddings_joined = np.concatenate(embeddings, axis=0)
burkhardt.obsm['X_SC'] = embeddings_joined

In [ ]:
# Run the UMAP on scanorama embeddings
sc.pp.neighbors(burkhardt, use_rep = "X_SC", n_neighbors=30)
sc.tl.umap(burkhardt)

In [ ]:
sc.tl.leiden(burkhardt, resolution=0.2, key_added='leiden')

In [ ]:
new_cluster_names = ['Alpha 1', 'Beta 1', 'Alpha 2', 'Beta 2', 'Delta', 'Beta 3']
burkhardt.rename_categories('leiden', new_cluster_names)

Rename the alpha and beta clusters to just 'Alpha' and 'Beta'

In [ ]:
burkhardt.obs['leiden'].cat.rename_categories({'Alpha 1': 'Alpha', 'Alpha 2': 'Alpha',
                                               'Beta 1': 'Beta', 'Beta 2': 'Beta',
                                               'Beta 3': 'Beta'}, inplace=True)

In [ ]:
# Save the data
burkhardt.write(burkhardt_data_directory + 'burkhardt21_merged.h5ad', compression='gzip')